In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from tabulate import tabulate

# 윈도우인 경우
from matplotlib import font_manager, rc

font_name = font_manager.FontProperties(fname='c:/Windows/Fonts/malgun.ttf').get_name()
rc('font',family=font_name)
plt.rcParams['axes.unicode_minus']=False

sns.set(rc = {'figure.figsize':(15,8)})

In [ ]:
pothole_df = pd.read_csv('data/인천/인천포트홀_0704.csv')

hum_df = pd.read_csv('데이터원소화/전체날씨데이터/일일_습도_평균_최대_최소_표준.csv')
_hum_df = pd.read_csv('데이터원소화/전체날씨데이터/어제_습도_평균_최대_최소_표준.csv')

temp_df = pd.read_csv('데이터원소화/전체날씨데이터/일일_기온_평균_최대_최소_표준.csv')
_temp_df = pd.read_csv('데이터원소화/전체날씨데이터/어제_기온_평균_최대_최소_표준.csv')

wind_df = pd.read_csv('데이터원소화/전체날씨데이터/일일_풍속_평균_최대_최소_표준.csv')
_wind_df = pd.read_csv('데이터원소화/전체날씨데이터/어제_풍속_평균_최대_최소_표준.csv')

aqua_df = pd.read_csv('데이터원소화/전체날씨데이터/일일_강수량_평균_최대_최소_표준.csv')
_aqua_df = pd.read_csv('데이터원소화/전체날씨데이터/어제_강수량_평균_최대_최소_표준.csv')

pothole_df['count']=1
pothole_df = pothole_df.groupby(['date','region_id'])[['count']].count()
pothole_df

count
date       region_id       
2021-08-11 2301058        1
           2304054        1
           2304055        1
           2304057        1
           2304058        3
...                     ...
2021-10-26 2305067        2
           2305072        1
           2305074        1
           2308051        1
           2308078        1

[622 rows x 1 columns]

 # (1) 포트홀과 강수량 

In [ ]:
aqua_days_df = pd.read_csv('데이터원소화/전체날씨데이터/2일_5일간_강수량_평균_최대_최소_표준.csv')
aqua_days_df = aqua_days_df.set_index(['date','region_id'])

merge_ad = pd.merge(pothole_df, aqua_days_df, how='inner', left_index=True, right_index=True)
# merge_ad = merge_ad.reset_index()
merge_ad['count'] = merge_ad['count'].fillna(0)
merge_ad.to_csv('data/인천포트홀강수량.csv')

In [ ]:
merge_ad_5d = merge_ad.groupby(['aqua_14d'],as_index=False).agg({'count':'count', 'aqua_14d_idx':'mean'})
# merge_ad_5d['count_idx']= merge_ad_5d['count'].apply(lambda x: x%15)
sns.relplot(x='count', y='aqua_14d',hue='aqua_14d_idx',data=merge_ad_5d)
sns.catplot(x='count', y='aqua_14d',data=merge_ad_5d, kind='swarm').fig.set_size_inches(10,5)


merge_ad_5d[merge_ad_5d['count']==100]

In [ ]:
merge_ad_d = merge_ad.groupby(['aqua_sum'],as_index=False).agg({'count':'count', 'aqua_sum_idx':'max'})
sns.relplot(x='count', y='aqua_sum',hue='aqua_sum_idx',data=merge_ad_d)

###

# (2) 포트홀과 기온 변화량

In [ ]:
temp_days_df = pd.read_csv('데이터원소화/전체날씨데이터/5일간_기온_평균최대_평균최소_최대일교차_최대기온_최소기온.csv')

temp_days_df = temp_days_df.set_index(['date','region_id'])

merge_td = pd.merge(pothole_df, temp_days_df, how='inner', left_index=True, right_index=True)

merge_td = merge_td.reset_index()

merge_td

In [ ]:
merge_td_tmax = merge_td.groupby('temp_max_5d',as_index=False).agg({'count':'count', 'temp_max_5d_idx':'mean'})
sns.relplot(x='count', y='temp_max_5d',hue='temp_max_5d_idx',data=merge_td_tmax)

In [ ]:
merge_td_tmin = merge_td.groupby('temp_min_5d',as_index=False).agg({'count':'count', 'temp_min_5d_idx':'mean'})
sns.relplot(x='count', y='temp_min_5d',hue='temp_min_5d_idx',data=merge_td_tmin)

In [ ]:
merge_td_chg = merge_td.groupby('temp_chg_max_5d',as_index=False).agg({'count':'count', 'temp_chg_max_5d_idx':'mean'})


sns.relplot(x='count', y='temp_chg_max_5d',hue='temp_chg_max_5d_idx',data=merge_td_chg)